# GPT2 Language Model Training

In this notebook, we will train our model on sample data to estimate its runnig time.


In [1]:
import os
import wandb
import torch
import random
import numpy as np
import pandas as pd
from rich import print
from pathlib import Path
from tqdm.auto import tqdm
from dotenv import load_dotenv
from collections import defaultdict

In [2]:
from datasets import load_dataset
import transformers
import huggingface_hub
from transformers import AutoConfig
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
sns.set_theme(context='notebook', style='dark')
tqdm.pandas()

In [5]:
# fix random seed
seed = 42

random.seed(seed)

np.random.seed(seed)

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

transformers.set_seed(seed)

In [6]:
dataset_path = Path('../data/wikipeda.csv')
tokenizer_pah = Path('../models/nano-gpt-tokenizer/')
model_ckpt = 'openai-community/gpt2'
model_name = 'arabic-nano-gpt'
model_path = Path(f'../models/{model_name}')

In [7]:
print(model_path)

../models/arabic-nano-gpt

In [8]:
# wandb configs
PROJECT_NAME = 'Arabic-Nano-GPT'
JOB_TYPE = 'LM-Modeling'
TAGS = ['MOdeling', 'Transformers', 'GPT2', 'Language-Modeling', 'Arabic']
NOTES = 'LM Training on Arabic Data using Nano GPT2 Model Architecture'
RUN_NAME = 'Arabic-NanoGPT-LM-on-40%-Wikipedia'
config = defaultdict(dict)

In [9]:
load_dotenv()
wandb.login()
huggingface_hub.login(token=os.getenv('HF_TOKEN'),
      add_to_git_credential=True, write_permission=True)  # HF

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: e_hossam96. Use `wandb login --relogin` to force relogin
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [10]:
# load the dataset
dataset = load_dataset(
    'csv', data_files='../data/wikipeda.csv', split='train[:500000]')
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 500000
})

In [11]:
dataset = dataset.train_test_split(train_size=0.9, seed=seed)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 450000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 50000
    })
})

In [12]:
test_data = dataset['test'].train_test_split(test_size=0.5, seed=seed)
dataset['valid'] = test_data['train']
dataset['test'] = test_data['test']
del test_data
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 450000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 25000
    })
    valid: Dataset({
        features: ['text'],
        num_rows: 25000
    })
})

In [13]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_pah)
print(tokenizer)

GPT2TokenizerFast(name_or_path='../models/nano-gpt-tokenizer', vocab_size=8192, model_max_length=1024, 
is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 
'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False),  
added_tokens_decoder={
        0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, 
special=True),
}

In [14]:
def tokenize(batch):
    outputs = tokenizer(
        batch['text'],
        truncation=True,
        return_overflowing_tokens=True,
    )
    return {'input_ids': outputs['input_ids']}


tokenized_dataset = dataset.map(
    tokenize, batched=True, remove_columns=dataset['train'].column_names
)
print(tokenized_dataset)

Parameter 'function'=<function tokenize at 0x7fccab6587c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/450000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 450006
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 25000
    })
    valid: Dataset({
        features: ['input_ids'],
        num_rows: 25000
    })
})

In [15]:
tokenizer.decode(tokenized_dataset['train'][0]['input_ids'])

'عيد النبي ابن عبد الله من طلعت \xa0 \xa0 \xa0 \xa0 \xa0 \xa0\xa0 شمس الهداية من قرآنه العلوي'

In [16]:
model_config = AutoConfig.from_pretrained(
    model_ckpt, vocab_size=tokenizer.vocab_size,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    n_positions=tokenizer.model_max_length,
    n_ctx=tokenizer.model_max_length,
    n_embd=256, n_head=4, n_layer=4,
)
print(model_config)

GPT2Config {
  "_name_or_path": "openai-community/gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 256,
  "n_head": 4,
  "n_inner": null,
  "n_layer": 4,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.45.2",
  "use_cache": true,
  "vocab_size": 8192
}

In [17]:
model = AutoModelForCausalLM.from_config(model_config)
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(8192, 256)
    (wpe): Embedding(1024, 256)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-3): 4 x GPT2Block(
        (ln_1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=768, nx=256)
          (c_proj): Conv1D(nf=256, nx=256)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=1024, nx=256)
          (c_proj): Conv1D(nf=256, nx=1024)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=256, out_features=8192, bias=False)
)

In [18]:
print('Size in MBs:', model.get_memory_footprint() / 1_000_000)

Size in MBs: 26.269712

In [19]:
print('Num Params:', model.num_parameters() / 1_000_000, 'M')

Num Params: 5.518848 M

In [20]:
num_epochs = 1
accum_steps = 8
batch_size = 32
total_steps = len(tokenized_dataset['train']) * \
    num_epochs // (batch_size * accum_steps)
total_steps, batch_size * accum_steps

(1757, 256)

In [21]:
training_args = TrainingArguments(
    output_dir=model_path,
    run_name=RUN_NAME,
    report_to='wandb',
    save_strategy='no',
    eval_strategy='steps',
    gradient_accumulation_steps=accum_steps,
    overwrite_output_dir=True,
    data_seed=seed, seed=seed,
    learning_rate=1e-4,
    weight_decay=0.0,
    warmup_ratio=0.0,
    eval_steps=175,
    logging_steps=10,
    log_level='error',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
)

In [22]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['valid'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [24]:
run = wandb.init(project=PROJECT_NAME, job_type=JOB_TYPE,
                 name=RUN_NAME, notes=NOTES, tags=TAGS, config=config)

In [25]:
_ = trainer.train()

Step,Training Loss,Validation Loss
175,7.519900,7.484433
350,6.935800,6.884676
525,6.665400,6.597342
700,6.477400,6.418382
875,6.345200,6.279194
1050,6.238000,6.178625
1225,6.173500,6.104737
1400,6.120400,6.052239
1575,6.087800,6.021203
1750,6.077700,6.008078


In [26]:
print(trainer.evaluate(tokenized_dataset['test'], metric_key_prefix='test'))

{
    'test_loss': 6.0123982429504395,
    'test_runtime': 129.2453,
    'test_samples_per_second': 193.431,
    'test_steps_per_second': 6.051,
    'epoch': 0.9995022399203584
}

In [27]:
wandb.finish()

eval/loss,█▅▄▃▂▂▁▁▁▁
eval/runtime,▁▇▇▇██▇▇▇▇
eval/samples_per_second,█▂▂▁▁▁▂▂▁▁
eval/steps_per_second,█▂▂▁▁▁▂▂▁▁
test/loss,▁
test/runtime,▁
test/samples_per_second,▁
test/steps_per_second,▁
train/epoch,▁▁▁▁▁▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▁▁▂▂▂▂▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇████
train/grad_norm,▇▆▅▁▁▁▄▃▃▄▅▄▇▄▅▄▄▅▄▄▆▇▆▆▆▆▆▅▅▅▆▆▆▆▆█▆▅▇█


In [28]:
trainer.save_model(model_path)

In [29]:
_ = trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/22.1M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]